## NLP Project - Sentiment Analysis: Hebrew

## Contents:

1. Exploratory Data Analysis (EDA)
2. Pre-Processing the data
3. Choosing ML algorithm for the task at hand 
4. Training Session
5. Predictions 
6. Saving results into csv file

In [129]:
!pip install hebrew_tokenizer

  Created wheel for hebrew-tokenizer: filename=hebrew_tokenizer-2.1.0-py3-none-any.whl size=12083 sha256=563bd7bfe1417b907eb666e4f9b2c88131d804b0d77cf33d2af5803c236e804b
  Stored in directory: /Users/henhalpert/Library/Caches/pip/wheels/5b/2c/ba/35bd5c37863121cb2b721aee7955a944f1a27949cdd3c81c87
Successfully built hebrew-tokenizer


In [242]:
# imports:
import numpy as np
import pandas as pd
import os
import nltk
import hebrew_tokenizer as ht

# ML: I will first try SVM as my ML algorithm 
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV # tuning hyperparameters
# from skopt import BayesSearchCV
from sklearn.naive_bayes import MultinomialNB

In [11]:
import hebrew_tokenizer as ht
a = []
hebrew_text = story.to_list()[0]
tokens = ht.tokenize(hebrew_text)  # tokenize returns a generator!
for grp, token, token_num, (start_index, end_index) in tokens:
    print('{}, {}'.format(grp, token))
    if grp == 'HEBREW':
        a.append(token)

HEBREW, זה
HEBREW, התחיל
HEBREW, בבוקר
PUNCTUATION, ,
HEBREW, היה
HEBREW, יום
HEBREW, חם
HEBREW, מאוד
PUNCTUATION, .
HEBREW, אני
HEBREW, יושב
HEBREW, בבית
HEBREW, עם
HEBREW, חברה
HEBREW, שלי
HEBREW, ופתאום
HEBREW, המזגן
HEBREW, מתקלקל
PUNCTUATION, .
HEBREW, בגלל
HEBREW, שזאת
HEBREW, דירה
HEBREW, שכורה
HEBREW, אני
HEBREW, מתקשר
HEBREW, לבעלת
HEBREW, הדירה
PUNCTUATION, ,
HEBREW, מספר
HEBREW, לה
HEBREW, על
HEBREW, המזגן
HEBREW, המקולקל
HEBREW, תוך
HEBREW, כדי
HEBREW, שאני
HEBREW, מתחיל
HEBREW, להזיע
HEBREW, מהמצח
PUNCTUATION, .
HEBREW, בעלת
HEBREW, הדירה
HEBREW, עונה
HEBREW, לי
HEBREW, בתשובה
HEBREW, שלפי
HEBREW, החוזה
HEBREW, שחתמתי
HEBREW, עליו
PUNCTUATION, ,
HEBREW, אני
HEBREW, צריך
HEBREW, לשלם
HEBREW, על
HEBREW, תיקון
HEBREW, המזגן
PUNCTUATION, .
HEBREW, התשובה
HEBREW, שלה
HEBREW, לא
HEBREW, הייתה
HEBREW, מקובלת
HEBREW, עלי
PUNCTUATION, ,
HEBREW, אז
HEBREW, עניתי
HEBREW, לה
HEBREW, שאם
HEBREW, ככה
HEBREW, אני
HEBREW, עוזב
HEBREW, את
HEBREW, הדירה
HEBREW, ומוצא
HEBREW, לעצמי
HEBREW, מ

In [12]:
a

['זה',
 'התחיל',
 'בבוקר',
 'היה',
 'יום',
 'חם',
 'מאוד',
 'אני',
 'יושב',
 'בבית',
 'עם',
 'חברה',
 'שלי',
 'ופתאום',
 'המזגן',
 'מתקלקל',
 'בגלל',
 'שזאת',
 'דירה',
 'שכורה',
 'אני',
 'מתקשר',
 'לבעלת',
 'הדירה',
 'מספר',
 'לה',
 'על',
 'המזגן',
 'המקולקל',
 'תוך',
 'כדי',
 'שאני',
 'מתחיל',
 'להזיע',
 'מהמצח',
 'בעלת',
 'הדירה',
 'עונה',
 'לי',
 'בתשובה',
 'שלפי',
 'החוזה',
 'שחתמתי',
 'עליו',
 'אני',
 'צריך',
 'לשלם',
 'על',
 'תיקון',
 'המזגן',
 'התשובה',
 'שלה',
 'לא',
 'הייתה',
 'מקובלת',
 'עלי',
 'אז',
 'עניתי',
 'לה',
 'שאם',
 'ככה',
 'אני',
 'עוזב',
 'את',
 'הדירה',
 'ומוצא',
 'לעצמי',
 'מחליף',
 'לחוזה',
 'על',
 'פי',
 'החוזה',
 'שחתמנו',
 'עליו',
 'בעלת',
 'הדירה',
 'לא',
 'אהבה',
 'את',
 'התשובה',
 'שלי',
 'ואמרה',
 'לי',
 'אוקיי',
 'וניתקה',
 'באותו',
 'הזמן',
 'הלכתי',
 'לבית',
 'של',
 'ההורים',
 'שלי',
 'בשביל',
 'ללמוד',
 'ולהיות',
 'במזגן',
 'לאחר',
 'כמה',
 'שעות',
 'אני',
 'חוזר',
 'לדירה',
 'ואני',
 'רואה',
 'את',
 'בעלת',
 'הדירה',
 'בתוך',
 'הדירה',
 'מוציאה',
 '

#### 1. EDA

In [243]:
df = pd.read_excel('annotated_corpus_for_train.xlsx', engine='openpyxl')

In [244]:
df.head()

,story,gender
0,"בוקר אחד קמתי סהרורי יצאתי מהמיטה קצת מטושטש ,...",m
1,לחבר שלי היה יום הולדת וחיפשנו מה אפשר לעשות ל...,m
2,"השנה האחרונה הייתה שנת קורונה, שנה לא פשוטה בק...",m
3,"לפני כחצי שנה עברתי לגור בצפון עם בת זוגתי, עב...",m
4,"יום חמישי רגיל, תמיד מתחיל לעבור טיפה מאוחר יו...",m


In [245]:
df.describe()
# there are 364 individuals in training set.

,story,gender
count,364,364
unique,364,2
top,לפני כ3שלושה חודשים כשבוע לאחר שראש הממשלה זאז...,m
freq,1,271


In [246]:
# how many males ? 271; how amny females ? 93 (see below)
# are the classes balanced ? There are more male individuals than females.

print(f" Males\n: {df[df['gender'] == 'm'].count()}")
print(f" Females\n: {df[df['gender'] == 'f'].count()}")

 Males
: story     271
gender    271
dtype: int64
 Females
: story     93
gender    93
dtype: int64


In [247]:
# Taking one story and its label for exploration purposes:

sample_example = df.sample(1)
story, label =  sample_example['story'], sample_example['gender']

for word in story:
    print(word)
print(label)

בשנה 2020, נגיף הקורונה התפרץ ברחבי העולם, אני שבדרך כלל מאמין במדע הייתי בדעה שזה סתם שטות, עוד וירוס שטותי כמו שפעת ולכן הסתובבת לרוב בלי מסכה, המשכתי לחיות כהרגלי ולא ממש ייחסתי חשיבות. 

כל פעם שראיתי איזה שוטר או חשבתי אולי יש מצב שיש משהו שמתי איזה מטפחת רק כדי שלא יגידו לי כלום. 

כשהסתובבתי ברחובות הכל היה דיי שוקק ולא הבנתי מה אומרים בתקשורת, מה זאת הסכנה הזאת!? 

המשכתי לצאת למסיבות מחתרתיות, גם כשהיה סגר הלכתי להורים לארוחות שישי איכשהו הצלחתי לחמוק מהמשטרה. 

זה היה החלק הקל שבסיפור, כמעט שנה החזקתי ככה, חיי כרגיל ולא ממש מבין על מה כל ההמולה, זה עוד זן של שפעת! 

עד שחזרתי יום אחד מבילוי בבית קפה עם כמה חברים, פתאום הרגשתי איזה צינון, בהתחלה באמת חשבתי שזה שטויות, הלכתי כרגיל להורים לארוחת שישי, כמה ימים אחר כך הם התקשרו ואמרו שעשו בדיקת קורונה והם חיוביים, מחכים לתוצאות של הבדיקה שסבתא (!) 

הם ששמרו על ההנחיות, לא מיהרו לצאת מהבית, אפילו את הקניות בסופר הם הזמינו! 

טוב הם אנשים מבוגרים, אמא שלי כבר בפנסיה אבא צריך לצאת בעוד 3-4 שנים.. 

אני לא מאמין לבדיקות של הקורונה ה

### 2.Pre-Processing

In [248]:
# I would like to shuffle the data first

df = df.sample(frac=1)

In [249]:
# splitting df into input and labels:
X_train = df['story']
y_train = df['gender']

# and testing set:
df_test = pd.read_excel('corpus_for_test.xlsx', engine='openpyxl')
X_test = df_test['story']
y_test = ... # This is unknown to us

0      כחלק ממסגרת ההתנדבות שלי במגלה אני הולך לפעמיי...
1      לפני שנה החלטתי שאני רוצה להיות טייס, התחלתי ל...
2      בתקופת הקורונה של תחילת החיסונים נגד קורונה, א...
3      כפי שכולם מכירים או שמעו מחברים עולם הדייטים ה...
4      אחת החוויות שהכי זכורות לי, זו החוויה בפרו בטי...
                             ...                        
151    לא הרבה יודעים אך אני מתעסק הרבה בשוק ההון ובמ...
152    כשהייתי בן 4 נולד לי אח קטן, אני חייב להודות ש...
153    הכל התחיל אי שם בגיל ההתבגרות, היתה לי חברה מו...
154    נסעתי עם חברים לים, היה יום חם ורצינו ללכת לנצ...
155    הייתי בכיתה י,  ואני והשכבה של יצאנו לטיול שנת...
Name: story, Length: 156, dtype: object

In [92]:
# Tokenizing the words:
from nltk.stem.porter import *

# I would like to convert each story into a list of words, tokenize each word, and then 
# remove stopwords:\

# i will use the example that was given in this assignment for the stopwords. 
# If the model does not perform well I will create my own stopwords based on frequency of each word. 

with open('stop_word_list_heb_example.txt') as f:
    stopwords = f.read().splitlines()

def story_to_words(story):
    '''
    Converting a story into list of tokenized words; all stop words are removed.
    
    Args:
    story (str) - a story of 1 individual
    
    returns: an array of words
    '''
    
    # Tokenizing hebrew words:
    list_of_words = []
    
    tokens = ht.tokenize(story) 
    for grp, token, token_num, (start_index, end_index) in tokens:
        if grp == 'HEBREW':
            list_of_words.append(token)
    
    # Removing stopwords:
    clean_arr = [word for word in list_of_words if word not in stopwords]
    
    # Stemming: for now I will skip stemming the words; the words are in hebrew so I am not sure if its needed. 
    # clean_arr = [PorterStemmer().stem(w) for w in clean_arr]
    
    return clean_arr

def preprocess_data(data_train, data_test):
    '''
    This function will convert each story into words using story_to_words() function, and put all 
    of them in an array. 
    Args:
    data_train(df)
    data_test(df)
    
    Returns:
    preprocessed trained and testing sets, and labels of trained set. 
    
    '''
    
    words_train = [story_to_words(story) for story in data_train]
    words_test = [story_to_words(story) for story in data_test]
    return words_train, words_test

In [93]:
# A simple example of story_to_words before I pre-process the entire training set:
hebrew_text = story.to_list()[0]
clean_arr = story_to_words(hebrew_text)
clean_arr

['בכמה',
 'האחרונות',
 'דירה',
 'לגור',
 'בת',
 'א',
 'כשהתחילה',
 'הקורונה',
 'התחילה',
 'שרשרת',
 'ארועים',
 'מבאסים',
 'קודם',
 'הפסקתי',
 'לעבוד',
 'ידעתי',
 'שמגיעה',
 'אבטלה',
 'רצף',
 'עבודה',
 'מתמיד',
 'בשנים',
 'האחרונות',
 'שעוד',
 'מתמיד',
 'האבטלה',
 'ודמי',
 'האבטלה',
 'שתמיד',
 'קיבלתי',
 'עבודות',
 'ביטוח',
 'לאומי',
 'ממציא',
 'סעיף',
 'להוריד',
 'ימי',
 'אבטלה',
 'הפעם',
 'מסתבר',
 'שמי',
 'שתובע',
 'אבטלה',
 'זכאי',
 'בשנה',
 'הראשונה',
 'ל',
 'אחוז',
 'ל',
 'אחוז',
 'ואחר',
 'זכאי',
 'מבחינת',
 'ימי',
 'אבטלה',
 'חלוף',
 'מהתביעה',
 'המקורית',
 'שחודשיים',
 'המדינה',
 'עלתה',
 'גל',
 'האבטלה',
 'זכאי',
 'לאבטלה',
 'ימי',
 'זכאות',
 'המשמעות',
 'שלשכת',
 'התעסוקה',
 'חודש',
 'שולחת',
 'ימי',
 'התייצבות',
 'וביטוח',
 'לאומי',
 'ישר',
 'מנכה',
 'לקח',
 'חצי',
 'שנזכרו',
 'להכליל',
 'בתוך',
 'הזכאים',
 'לאבטלה',
 'מתוקף',
 'סעיף',
 'מסוים',
 'חירום',
 'התחלתי',
 'הכשרה',
 'מסוימת',
 'מכיוון',
 'מצאתי',
 'עבודה',
 'וחודש',
 'התחלתי',
 'אבטלה',
 'הודיעו',
 'לפנות',
 'הדיר

In [258]:
# Now can finally preprocess all data:

X_train_processed, X_test_processed = preprocess_data(X_train, X_test)

#### Extracting Bag-Of-Words Features:

For the model I will be implementing, rather than using the stories directly, we are going to transform each story into a Bag-of-Words feature representation. Keep in mind that 'in the wild' we will only have access to the training set so our transformer can only use the training set to construct a representation.

In [282]:
from sklearn.feature_extraction.text import CountVectorizer

def extract_BoW_features(words_train, words_test, vocabulary_size=800):
    """
    Extract Bag-of-Words for a given set of documents, already preprocessed into words.
    
    Returns BoW for both training and testing sets, and vocabulary of words. 
    """
    vectorizer = CountVectorizer(max_features=vocabulary_size,
                preprocessor=lambda x: x, tokenizer=lambda x: x)
    features_train = vectorizer.fit_transform(words_train).toarray()

    # Apply the same vectorizer to transform the test set
    features_test = vectorizer.transform(words_test).toarray()
    vocabulary = vectorizer.vocabulary_
    
    return features_train, features_test, vocabulary

In [283]:
# extracting BoW features: 
X_train_bow, X_test_bow, vocabulary = extract_BoW_features(X_train_processed, X_test_processed)

In [284]:
X_train_bow.shape

(364, 800)

In [285]:
# Lets take a look at the vocabulary: 
vocabulary

{'מ': 558,
 'הרגשתי': 273,
 'בגדים': 55,
 'למצוא': 512,
 'עלי': 675,
 'בעקבות': 105,
 'בחוץ': 67,
 'יוצא': 387,
 'חברים': 348,
 'סגור': 648,
 'בנוסף': 92,
 'שהייתי': 754,
 'הצלחתי': 260,
 'להכיר': 464,
 'חיי': 367,
 'בתוך': 127,
 'הלכתי': 224,
 'אוכל': 4,
 'רבות': 722,
 'גבוה': 135,
 'נשימה': 645,
 'אליה': 22,
 'הגעתי': 172,
 'נכנסתי': 636,
 'והייתי': 312,
 'לאכול': 435,
 'בעיקר': 100,
 'מים': 586,
 'קטנה': 703,
 'עשיתי': 690,
 'הליכה': 220,
 'שישי': 765,
 'הבוקר': 157,
 'ועד': 336,
 'השעה': 283,
 'בבוקר': 53,
 'מידי': 584,
 'לאורך': 431,
 'לבית': 444,
 'בעיר': 101,
 'לאט': 433,
 'התחלתי': 290,
 'חודש': 352,
 'שלושה': 772,
 'חודשים': 354,
 'להרגיש': 469,
 'עצמי': 683,
 'חזרה': 364,
 'קל': 708,
 'פתאום': 694,
 'מהבית': 570,
 'לטייל': 487,
 'ובנוסף': 305,
 'בחברה': 63,
 'רבים': 723,
 'להוריד': 457,
 'בי': 75,
 'בתור': 128,
 'ילד': 393,
 'לבן': 445,
 'במיוחד': 86,
 'טעים': 382,
 'אמרו': 31,
 'ואף': 302,
 'חשבתי': 374,
 'אמא': 27,
 'מבית': 564,
 'הספר': 249,
 'לעצמי': 529,
 'בחרתי': 74,
 '

In [286]:
# Finallly, I will want to create a validation set as well from the training set:
# lets make it 90% for training, 10% for validation:
n = X_train_bow.shape[0] # 364 entries in training set.
n_training = int(0.9 * n) # 90% goes to training. 

train_X = X_train_bow[:n_training]
val_X = X_train_bow[n_training:]

train_y = y_train[:n_training]
val_y= y_train[n_training:]

test_y = ... # unknown 
test_X = X_test_bow

### 3+4. ML Model and Training

In [287]:
# Training on 90%: 
parameters_for_SVM = {'kernel':('linear', 'rbf', 'poly', 'sigmoid'),
              'C':[10, 800],
             'degree':[1,3],
             'gamma': ('scale', 'auto')}


parameters_for_NB = {'alpha':(1.0, 0.0),
                    }
# init
svc = SVC()
clf = GridSearchCV(svc, parameters)
#training
clf.fit(train_X, train_y)
model = clf.best_estimator_

In [164]:
# Save the trained model
import joblib
joblib.dump(model, "model.joblib")

['model.joblib']

In [294]:
# for svm
predictions = model.predict(val_X)
predictions

array(['m', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm',
       'm', 'm', 'm', 'm', 'm', 'm', 'm', 'f', 'm', 'm', 'm', 'f', 'm',
       'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'f', 'm'],
      dtype=object)

In [304]:
# for bayes:
predictions_bayes = clf.predict(val_X)
predictions_bayes

array(['m', 'm', 'f', 'm', 'f', 'm', 'm', 'f', 'm', 'm', 'm', 'm', 'm',
       'm', 'm', 'm', 'm', 'm', 'f', 'm', 'f', 'f', 'm', 'm', 'f', 'm',
       'm', 'm', 'm', 'm', 'm', 'f', 'm', 'm', 'm', 'm', 'm'], dtype='<U1')

In [305]:
val_y.to_list()

['m',
 'm',
 'f',
 'm',
 'f',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'f',
 'f',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'm',
 'f',
 'm',
 'm',
 'm',
 'm',
 'f']

## 4.1 Naive Bayes Method

In [301]:
#init
clf = MultinomialNB()

clf.fit(train_X, train_y)
predictions_bayes = clf.predict(val_X)

In [302]:
# Metrics:
# for svm
from sklearn.metrics import classification_report
print(classification_report(val_y, predictions, target_names=target_names))

# for bayes:
print(classification_report(val_y, predictions_bayes, target_names=target_names))

              precision    recall  f1-score   support

           m       0.33      0.17      0.22         6
           f       0.85      0.94      0.89        31

    accuracy                           0.81        37
   macro avg       0.59      0.55      0.56        37
weighted avg       0.77      0.81      0.78        37

              precision    recall  f1-score   support

           m       0.50      0.67      0.57         6
           f       0.93      0.87      0.90        31

    accuracy                           0.84        37
   macro avg       0.72      0.77      0.74        37
weighted avg       0.86      0.84      0.85        37

